In [6]:
import pandas as pd 
import numpy as np 

import recordlinker

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
iowa_matches = pd.read_csv('/Users/kailinlu/Desktop/QMSSWork/RecordLinking/recordlinker/recordlinker/data/iowa_matches.csv')
iowa_nonmatches = pd.read_csv('/Users/kailinlu/Desktop/QMSSWork/RecordLinking/recordlinker/recordlinker/data/iowa_nonmatches.csv')

In [4]:
iowa_matches.head()

,uid1915,fname1915,lname1915,fullname1915,yob1915,hhid,fname1940,lname1940,fullname1940,yob1940,uid-hhid
0,uid0910071227,donald d,cutler,donald d cutler,1911,19067,donald dean,cutler,donald dean cutler,1911,uid0910071227-19067
1,uid0063131339,homer,taylor,homer taylor,1912,71505,homer ellis,taylor,homer ellis taylor,1912,uid0063131339-71505
2,uid0044088276,earl,stearnes,earl stearnes,1899,109708,earl,stearns,earl stearns,1900,uid0044088276-109708
3,uid0067053130,theodore,hornaday,theodore hornaday,1904,108304,theodore i,harnaday,theodore i harnaday,1904,uid0067053130-108304
4,uid0066046148,jack r,turner,jack r turner,1907,105092,jack r,turner,jack r turner,1907,uid0066046148-105092


In [239]:
ORIG_LENGTH = 20 
BATCH_SIZE = 32
LATENT_DIM = 16
ENCODE_DIM = [128, 128] 
DECODE_DIM = [128, 128]
LR = 1e-4
EPOCHS=20

namesA = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1915',
                                                     max_length=ORIG_LENGTH, embed_type='letters')
namesB = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1940',
                                                     max_length=ORIG_LENGTH, embed_type='letters')

vae = recordlinker.model.VAE(batch_size=BATCH_SIZE,
                             orig_dim=ORIG_LENGTH, 
                             latent_dim=LATENT_DIM,
                             encode_dim=ENCODE_DIM,
                             decode_dim=DECODE_DIM,
                             lr=LR)
vae.train(namesA, namesB, 
          epochs=EPOCHS, 
          run_id='test_run_1',
          save_path='/Users/kailinlu/Desktop/QMSSWork/RecordLinking/models/test_model',
          tensorboard=True, 
          earlystop=False, 
          reconstruct=False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (32, 20)             0                                            
__________________________________________________________________________________________________
enc_0 (Dense)                   (32, 128)            2688        input_1[0][0]                    
__________________________________________________________________________________________________
enc_1 (Dense)                   (32, 128)            16512       enc_0[0][0]                      
__________________________________________________________________________________________________
mu (Dense)                      (32, 16)             2064        enc_1[0][0]                      
__________________________________________________________________________________________________
log_sigma 

In [237]:
namesA_3d = namesA[:,:,np.newaxis]
namesB_3d = namesB[:,:,np.newaxis]

EPOCHS = 100
LR = .0001

conv_vae = recordlinker.model.ConvolutionalVAE(batch_size=BATCH_SIZE, 
                                               orig_dim=ORIG_LENGTH, 
                                               latent_dim=8, 
                                               lr=LR)
conv_vae.train(namesA_3d, namesB_3d, 
              epochs=EPOCHS, 
              run_id='test_run_5',
              optimizer='rmsprop',
              save_path='/Users/kailinlu/Desktop/QMSSWork/RecordLinking/models/test_conv',
              tensorboard=True, 
              earlystop=False, 
              reconstruct=False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (32, 20, 1)          0                                            
__________________________________________________________________________________________________
conv0 (Conv1D)                  (32, 10, 16)         64          input_1[0][0]                    
__________________________________________________________________________________________________
pool0 (MaxPooling1D)            (32, 8, 16)          0           conv0[0][0]                      
__________________________________________________________________________________________________
conv1 (Conv1D)                  (32, 4, 32)          1568        pool0[0][0]                      
__________________________________________________________________________________________________
pool1 (Max

3456/3456 [==============================] - 0s 91us/step - loss: 3.3194e-07 - acc: 0.3163 - val_loss: 3.3582e-07 - val_acc: 0.3535
Epoch 32/100
3456/3456 [==============================] - 0s 91us/step - loss: 3.3173e-07 - acc: 0.3140 - val_loss: 3.3578e-07 - val_acc: 0.3514
Epoch 33/100
3456/3456 [==============================] - 0s 88us/step - loss: 3.3170e-07 - acc: 0.3160 - val_loss: 3.3578e-07 - val_acc: 0.3553
Epoch 34/100
3456/3456 [==============================] - 0s 88us/step - loss: 3.3170e-07 - acc: 0.3177 - val_loss: 3.3579e-07 - val_acc: 0.3539
Epoch 35/100
3456/3456 [==============================] - 0s 85us/step - loss: 3.3170e-07 - acc: 0.3165 - val_loss: 3.3579e-07 - val_acc: 0.3600
Epoch 36/100
3456/3456 [==============================] - 0s 89us/step - loss: 3.3170e-07 - acc: 0.3187 - val_loss: 3.3578e-07 - val_acc: 0.3567
Epoch 37/100
3456/3456 [==============================] - 0s 85us/step - loss: 3.3170e-07 - acc: 0.3181 - val_loss: 3.3580e-07 - val_acc: 0.359

3456/3456 [==============================] - 0s 92us/step - loss: 3.3225e-07 - acc: 0.3187 - val_loss: 3.3579e-07 - val_acc: 0.3530
Epoch 88/100
3456/3456 [==============================] - 0s 90us/step - loss: 3.3249e-07 - acc: 0.3174 - val_loss: 3.3948e-07 - val_acc: 0.3532
Epoch 89/100
3456/3456 [==============================] - 0s 86us/step - loss: 3.3253e-07 - acc: 0.3179 - val_loss: 3.3579e-07 - val_acc: 0.3558
Epoch 90/100
3456/3456 [==============================] - 0s 80us/step - loss: 3.3249e-07 - acc: 0.3145 - val_loss: 3.4319e-07 - val_acc: 0.3561
Epoch 91/100
3456/3456 [==============================] - 0s 85us/step - loss: 3.3246e-07 - acc: 0.3184 - val_loss: 3.3577e-07 - val_acc: 0.3591
Epoch 92/100
3456/3456 [==============================] - 0s 88us/step - loss: 3.3229e-07 - acc: 0.3171 - val_loss: 3.3577e-07 - val_acc: 0.3549
Epoch 93/100
3456/3456 [==============================] - 0s 98us/step - loss: 3.3253e-07 - acc: 0.3155 - val_loss: 3.3948e-07 - val_acc: 0.359

In [265]:
lstm_vae = recordlinker.model.LSTMVAE(batch_size=BATCH_SIZE, 
                                      timesteps=ORIG_LENGTH, 
                                      orig_dim=27,
                                      latent_dim=2, 
                                      lr=LR) 
lstm_vae._build_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (32, 20, 27)         0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (32, 20, 64)         23552       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (32, 20, 64)         33024       lstm_1[0][0]                     
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (32, 2)              536         lstm_2[0][0]                     
__________________________________________________________________________________________________
mu (Dense)

(<keras.engine.training.Model at 0x130b72b00>,
 <bound method VariationalLoss.vae_loss of <recordlinker.model.VariationalLoss object at 0x133c7af28>>)